In [3]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

UA = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# -------------------------
# 인코딩 감지 & 디코딩
# -------------------------
_META_CHARSET_RE = re.compile(br'charset\s*=\s*["\']?([\w\-]+)', re.I)

def _smart_text(resp: requests.Response, fallback_domain_hint: str | None = None) -> str:
    """
    응답 바이트에서 meta/http 헤더의 charset을 우선 반영하고,
    없으면 apparent_encoding을 쓰고, 마지막으로 도메인 힌트로 보정.
    """
    raw = resp.content
    enc = None

    # 1) HTTP 헤더 우선
    ct = resp.headers.get("Content-Type", "")
    m = re.search(r'charset=([\w\-]+)', ct, flags=re.I)
    if m:
        enc = m.group(1).lower()

    # 2) <meta ... charset=...> 스캔
    if not enc:
        m2 = _META_CHARSET_RE.search(raw[:4096])  # 초반부만 보면 충분
        if m2:
            enc = m2.group(1).decode("ascii", "ignore").lower()

    # 3) requests의 추정
    if not enc:
        enc = (resp.apparent_encoding or "").lower() or None

    # 4) 도메인 힌트 (KIND 기본 cp949)
    if not enc and fallback_domain_hint and "kind.krx.co.kr" in fallback_domain_hint.lower():
        enc = "cp949"

    # 5) 표준화
    if enc in ("euc-kr", "ks_c_5601-1987", "ks_c_5601-1989"):
        enc = "cp949"

    try:
        return raw.decode(enc or "utf-8", errors="strict")
    except Exception:
        # 최후의 보루: cp949 → utf-8 순서로 시도
        for e in (enc, "cp949", "utf-8", "latin1"):
            if not e:
                continue
            try:
                return raw.decode(e, errors="strict")
            except Exception:
                pass
        # 그래도 실패하면 느슨하게
        return raw.decode(enc or "utf-8", errors="replace")

def _get(url: str, timeout: int = 20, hint: str | None = None) -> str:
    r = requests.get(url, timeout=timeout, headers=UA)
    r.raise_for_status()
    return _smart_text(r, fallback_domain_hint=hint or url)

# -------------------------
# 태그 제거 유틸
# -------------------------
def _strip_tags(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    for t in soup(["script", "style", "noscript"]): t.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")
    for p in soup.find_all("p"):
        p.insert_before("\n"); p.insert_after("\n")
    text = soup.get_text("\n", strip=True)
    text = re.sub(r"\r\n?", "\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def fetch_viewer_html(url: str, timeout: int = 20) -> str:
    return _get(url, timeout=timeout, hint=url)

# -------------------------
# KIND: acptNo/docNo → searchContents → setPath(...)에서 docLocPath 추출
# -------------------------
def _kind_pick_acpt_doc(viewer_html: str) -> tuple[str | None, str | None]:
    soup = BeautifulSoup(viewer_html, "lxml")
    acpt = None
    el = soup.select_one("#acptNo")
    if el and el.has_attr("value"):
        acpt = el["value"].strip() or None

    doc = None
    opt = soup.select_one("#mainDoc option[selected]") or soup.select_one("#mainDoc option:nth-of-type(2)")
    if opt and opt.get("value"):
        doc = opt["value"].split("|", 1)[0].strip() or None
    return acpt, doc

def _kind_fetch_doclocpath(kind_base_url: str, doc_no: str, timeout: int = 20) -> str | None:
    sc_url = urljoin(kind_base_url, f"/common/disclsviewer.do?method=searchContents&docNo={doc_no}")
    text = _get(sc_url, timeout=timeout, hint=kind_base_url)

    # setPath('tocLocPath','docLocPath','server',...)
    m = re.search(r"setPath\(\s*['\"][^'\"]*['\"]\s*,\s*['\"]([^'\"]+)['\"]\s*,", text)
    if m:
        return m.group(1).strip()
    # 백업: setPath2(...)
    m2 = re.search(r"setPath2\(\s*['\"][^'\"]*['\"]\s*,\s*['\"]([^'\"]+)['\"]\s*,", text)
    return m2.group(1).strip() if m2 else None

def _kind_fetch_text_from_docloc(kind_base_url: str, doc_loc_path: str, timeout: int = 20) -> str:
    doc_url = urljoin(kind_base_url, doc_loc_path)
    html = _get(doc_url, timeout=timeout, hint=kind_base_url)
    return _strip_tags(html)

# -------------------------
# iframe 백업 로직 (DART/KIND 공통)
# -------------------------
def _find_iframe_src(viewer_html: str) -> str | None:
    soup = BeautifulSoup(viewer_html, "lxml")
    iframe = soup.find("iframe", id="docViewFrm")
    if iframe:
        src = (iframe.get("src") or "").strip()
        if src:
            return src
    m = re.search(r'docViewFrm\s*\.src\s*=\s*[\'"]([^\'"]+)[\'"]', viewer_html, flags=re.I)
    if m:
        return m.group(1).strip()
    m2 = re.search(r"setPath\(\s*['\"][^'\"]*['\"]\s*,\s*['\"]([^'\"]+)['\"]", viewer_html)
    if m2:
        return m2.group(1).strip()
    return None

def _fetch_iframe_text(viewer_html: str, viewer_url: str, timeout: int = 20) -> str:
    src = _find_iframe_src(viewer_html)
    if not src:
        return ""
    iframe_url = urljoin(viewer_url, src)
    html = _get(iframe_url, timeout=timeout, hint=viewer_url)
    return _strip_tags(html)

# -------------------------
# 최종 진입점
# -------------------------
def extract_disclosure_text(url: str, timeout: int = 20) -> str:
    viewer_html = fetch_viewer_html(url, timeout=timeout)

    if "kind.krx.co.kr" in url.lower():
        acpt, doc = _kind_pick_acpt_doc(viewer_html)
        if doc:
            doc_loc = _kind_fetch_doclocpath(url, doc, timeout=timeout)
            if doc_loc:
                text = _kind_fetch_text_from_docloc(url, doc_loc, timeout=timeout)
                if text and len(text) > 50:
                    return text
        return _fetch_iframe_text(viewer_html, url, timeout=timeout)

    # DART 등은 기본적으로 iframe 따라가기 (DART도 EUC-KR 페이지가 있음 → _get 사용)
    return _fetch_iframe_text(viewer_html, url, timeout=timeout)



In [4]:
kind_url = "https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20210105000343"
print("KIND:", extract_disclosure_text(kind_url)[:1500], "\n---\n")

KIND: :: 70192_기타 경영사항(특허권 취득)(자율공시)
기타 경영사항(특허권 취득)(자율공시)
1. 특허명칭
의료기관 픽업 방법 및 의료기관 픽업 시스템
2. 특허 주요내용
1. 검진 예약 시스템(ex.에버헬스)과 이동수단 연계 시스템(ex : 택시호출 서비스)을 이용하여, 검진 예약 시 원격지 또는 거동 불편, 노령 이용자를 위해 픽업 서비스까지 함께 이용할 수 있도록 연계하는 시스템
2. 이용자를 픽업한 후 검진기관으로 이동 시 실시간 확인이 가능하고, 픽업 및 도착관련 문자도 이용자에게 전송하며, 검진기관은 이용자의 도착시간을 인지함으로써 신속한 대응 가능
3. 건강검진 접근이 어려운 고령자, 원격지 부모님 또는 거동이 불편한 분들도 안심하고 손쉽게 검진을 받을 수 있음.
4. 이용자가 검진기관에서 검사를 마치기 전 검진기관內 시스템을 통해 검사 완료 예상 시간을 확인하여 이동수단을 재호출하고, 이용자를 픽업한 후 원하는 도착지까지 이동
3. 특허권자
㈜유비케어
4. 특허취득일자
2021-01-05
5. 특허 활용계획
1. 검진사업(에버헬스)의 외부 경쟁력 강화와 프리미엄급 서비스 제공을 통한 수수료 증대
2. 비대면이 필요한 (긴급)의료 상황에서 당뇨병 등의 만성질환 환자가"의사랑 EMR"을 사용하는 병의원을 통하여 검사를 요청하면 병의원에서는 "의사랑 EMR"과 픽업시스템의 연계를 통하여 환자에게 검사키트를 보내고 이를 회수하며, 검사기관에 전달하는 것까지도 진행 가능
6. 확인일자
2021-01-05
7. 기타 투자판단에 참고할 사항
- 국내 특허
- 특허 취득일자 및 확인일자는 특허 등록료 납부일
- 특허 출원번호 : 10-2018-0107631 
---



In [6]:
import pandas as pd
import os

# 현재 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 디렉토리: {current_dir}")

# 입력/출력 파일 경로 (절대 경로로 설정)
base_dir = "/Users/taechanan/dart-sentiment-analysis/dags/data"
in_path = os.path.join(base_dir, "kind_disclosures_20250822_070850.csv")  # 원본 CSV
out_path = os.path.join(base_dir, "kind_disclosures_20250822_070850.with_text.csv")  # 저장할 CSV

print(f"입력 파일 경로: {in_path}")
print(f"출력 파일 경로: {out_path}")

# 파일 존재 확인
if not os.path.exists(in_path):
    raise FileNotFoundError(f"입력 파일을 찾을 수 없습니다: {in_path}")

# CSV 읽기 (인코딩 가볍게 시도)
df = None
for enc in ("utf-8-sig", "utf-8", "cp949", "euc-kr"):
    try:
        df = pd.read_csv(in_path, encoding=enc)
        print(f"CSV 파일을 {enc} 인코딩으로 성공적으로 읽었습니다.")
        break
    except Exception as e:
        print(f"{enc} 인코딩 시도 실패: {e}")
        pass
if df is None:
    raise ValueError("CSV를 읽을 수 없습니다. 인코딩을 확인하세요.")

# detail_url → raw_text 생성 (extract_disclosure_text는 이미 정의되어 있다고 가정)
if "detail_url" not in df.columns:
    raise ValueError("CSV에 'detail_url' 컬럼이 없습니다.")

df["raw_text"] = df["detail_url"].fillna("").astype(str).apply(
    lambda u: extract_disclosure_text(u) if u else ""
)

# 저장
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("saved ->", out_path)


현재 디렉토리: /Users/taechanan/dart-sentiment-analysis/analysis
입력 파일 경로: /Users/taechanan/dart-sentiment-analysis/dags/data/kind_disclosures_20250822_070850.csv
출력 파일 경로: /Users/taechanan/dart-sentiment-analysis/dags/data/kind_disclosures_20250822_070850.with_text.csv
CSV 파일을 utf-8-sig 인코딩으로 성공적으로 읽었습니다.
saved -> /Users/taechanan/dart-sentiment-analysis/dags/data/kind_disclosures_20250822_070850.with_text.csv


In [7]:
pd.read_csv(out_path)

,disclosed_at,company_name,stock_code,short_code,market,title,disclosure_id,detail_url,raw_text
0,2022-06-30 18:48,이원컴포텍,KR7088290002,88290.0,KOSDAQ,유상증자결정(자율공시)(종속회사의 주요경영사항)(제3자배정),20220630001079,https://kind.krx.co.kr/common/disclsviewer.do?...,:: 71776_유상증자결정(자율공시)(종속회사의 주요경영사항)\n유상증자결정(자율...
1,2022-06-30 18:24,코아시아씨엠,KR7196450001,196450.0,KOSDAQ,증권 발행결과(자율공시),20220630001078,https://kind.krx.co.kr/common/disclsviewer.do?...,:: 70065_증권 발행결과(자율공시)\n증권 발행결과(자율공시)\n1. 증권의 ...
2,2022-06-30 18:14,코아시아,KR7045970001,45970.0,KOSDAQ,주권 관련 사채권의 취득결정,20220630001053,https://kind.krx.co.kr/common/disclsviewer.do?...,:: 71988_주권 관련 사채권의 취득결정\n주권 관련 사채권의 취득결정\n1. ...
3,2022-06-30 18:14,코아시아씨엠,KR7196450001,196450.0,KOSDAQ,타법인주식및출자증권취득결정,20220630001052,https://kind.krx.co.kr/common/disclsviewer.do?...,:: 71381_타법인 주식 및 출자증권 취득결정\n타법인 주식 및 출자증권 취득결...
4,2022-06-30 18:14,코아시아씨엠,KR7196450001,196450.0,KOSDAQ,전환사채권발행결정(제10회차),20220630001036,https://kind.krx.co.kr/common/disclsviewer.do?...,주요사항보고서 / 거래소 신고의무 사항\n금융위원회 / 한국거래소 귀중\n2022 ...
...,...,...,...,...,...,...,...,...,...
495,2022-06-29 17:34,골프존데카,NaN,NaN,KONEX,주식등의대량보유상황보고서(일반),20220629000784,https://kind.krx.co.kr/common/disclsviewer.do?...,주식등의 대량보유상황보고서\n(일반서식 : 자본시장과 금융투자업에 관한 법률 제14...
496,2022-06-29 17:31,GS,KR7078930005,78930.0,KOSPI,주식등의대량보유상황보고서(일반),20220629000780,https://kind.krx.co.kr/common/disclsviewer.do?...,주식등의 대량보유상황보고서\n(일반서식 : 자본시장과 금융투자업에 관한 법률 제14...
497,2022-06-29 17:27,오늘이엔엠,KR7192410009,192410.0,KOSDAQ,주식등의대량보유상황보고서(일반),20220629000776,https://kind.krx.co.kr/common/disclsviewer.do?...,주식등의 대량보유상황보고서\n(일반서식 : 자본시장과 금융투자업에 관한 법률 제14...
498,2022-06-29 17:26,케이피에프,KR7024880007,24880.0,KOSDAQ,타인에대한채무보증결정,20220629000770,https://kind.krx.co.kr/common/disclsviewer.do?...,:: 71314_타인에 대한 채무보증 결정\n타인에 대한 채무보증 결정\n1. 채무...
